The following code processes monthly ERA5 precipitation and evaporation data from the Copernicus Climate Data Store, producing the following files used in the analysis notebooks: ERA5_PE_trendmap_coefs.nc, ERA5_PE_trendmap_pvals.nc, and ERA5_PE_timeseries_monthly.nc. 

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
xr.set_options(display_style='html')
import netCDF4
from scipy.stats import linregress
import os 
import dask
import cftime
import xesmf as xe

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.ticker as mticker
import matplotlib.path as mpath

import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature

import intake
from xmip.preprocessing import rename_cmip6
from xmip.preprocessing import broadcast_lonlat
from xmip.preprocessing import combined_preprocessing
from xmip.utils import google_cmip_col
from xmip.postprocessing import match_metrics
from xmip.postprocessing import merge_variables
from xmip.postprocessing import interpolate_grid_label
from xmip.postprocessing import concat_experiments
from xmip.postprocessing import pick_first_member

%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

In [2]:
# Directory for source data on Stanford Sherlock computing cluster 
# (must replace with your own directory if reproducing)

os.chdir('/oak/stanford/groups/earlew/zkaufman/Archive_KaufmanGRL2025/ERA5')

In [3]:
# get P-E data, edit the time dimension, edit lat lon dimension names

ERA5_data = xr.open_dataset('ERA5_monthly_P_E_197901_202112.nc')

ERA5_data['time'] = xr.cftime_range\
(start='1979', periods=len(ERA5_data.time), freq='M', calendar='noleap')

ERA5_data = ERA5_data.rename({'latitude':'lat','longitude':'lon'})
ERA5_data

<xarray.Dataset> Size: 4GB
Dimensions:  (lon: 1440, lat: 361, time: 516)
Coordinates:
  * lon      (lon) float32 6kB -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * lat      (lat) float32 1kB 0.0 -0.25 -0.5 -0.75 ... -89.5 -89.75 -90.0
  * time     (time) object 4kB 1979-01-31 00:00:00 ... 2021-12-31 00:00:00
Data variables:
    e        (time, lat, lon) float64 2GB ...
    tp       (time, lat, lon) float64 2GB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-04-01 23:44:38 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [4]:
# Convert units and subset to Southern Hemisphere for mapping 

unit_conversion_ERA5map = 1000 / 86400  # mday^-1 to kgm^-2s^-1 

tp = ERA5_data.tp.squeeze()*unit_conversion_ERA5map
e = ERA5_data.e.squeeze()*unit_conversion_ERA5map
PE_mapunits = tp - e


def subset_bylatitude(data, south_bound, north_bound):
    data_SO = data.sel(lat=slice(south_bound, north_bound))
    return data_SO

PE_subset = subset_bylatitude(PE_mapunits,-30,-90)

In [6]:
# calculate time trend regression coefficients and p values for annual mean (P-E) in each grid cell 


regression_coefficients_ERA5 = PE_subset[0,:,:].copy()
p_values_ERA5 = PE_subset[0,:,:].copy()

time = np.linspace(1990,2021,32)
# convert units, annually average, and time subset 
PE_SO_obs = PE_subset.sel(time=slice('1990','2021')).coarsen(time=12).mean()

for lat in PE_SO_obs.lat:
    for lon in PE_SO_obs.lon:
        data_slice = PE_SO_obs.sel(lat=lat, lon=lon)
            
        # Skip grid cells where the input values are 0 or nan at all timesteps
        if np.all(np.isnan(data_slice)) or np.all(data_slice == 0):
            continue

        # Calculate regression coefficients and p-values
        slope, intercept, r_value, p_value, std_err = linregress(time, data_slice)

        regression_coefficients_ERA5.loc[dict(lat=lat, lon=lon)] = slope
        p_values_ERA5.loc[dict(lat=lat, lon=lon)] = p_value
    
linear_regression_results_ERA5 = {}
linear_regression_results_ERA5['ERA5'] = regression_coefficients_ERA5
linear_regression_results_ERA5['ERA5_p_values'] = p_values_ERA5

In [8]:
# save postprocessed files for P-E trendmap regression coefficients and p values. 

os.chdir('/oak/stanford/groups/earlew/zkaufman/Archive_KaufmanGRL2025/postprocessed_analysis_notebooks')

output_filename = 'ERA5_PE_trendmap_coefs.nc'
linear_regression_results_ERA5['ERA5'].to_netcdf(output_filename)

output_filename = 'ERA5_PE_trendmap_pvals.nc'
linear_regression_results_ERA5['ERA5_p_values'].to_netcdf(output_filename)

In [9]:
# to obtain Southern Ocean integrated P-E, we need grid area and land mask.
# We obtain these from an arbitrary CMIP6 model on the intake_esm datastore

col = google_cmip_col()
df_base = col.search(
    table_id = ['fx'],
    variable_id = ['sftlf','areacella'],
    grid_label = ['gn'],
    experiment_id = ['piControl'],
    member_id = ['r1i1p1f1'],
    source_id = ["ACCESS-ESM1-5"]
)


# create xarray dictionaries for each search query 
kwargs = {
    'zarr_kwargs':{
        'consolidated':True,
        'use_cftime':True
    },
    'aggregate':False,
    
    'preprocess':combined_preprocessing
}

with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    grid_metrics_dict = df_base.to_dataset_dict(**kwargs)


grid_metrics_keys = list(grid_metrics_dict.keys())
print(grid_metrics_keys)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


['CMIP.CSIRO.ACCESS-ESM1-5.piControl.r1i1p1f1.fx.sftlf.gn.gs://cmip6/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl/r1i1p1f1/fx/sftlf/gn/v20191214/.20191214', 'CMIP.CSIRO.ACCESS-ESM1-5.piControl.r1i1p1f1.fx.areacella.gn.gs://cmip6/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl/r1i1p1f1/fx/areacella/gn/v20191214/.20191214']


In [17]:
# Regrid ERA5 P-E data (in map units) to the grid_metrics data

grid_metrics_grid = grid_metrics_dict[grid_metrics_keys[0]]

def regrid_dataarray(in_grid,out_grid):
    regridder = xe.Regridder(in_grid,out_grid, 'bilinear', periodic=True, ignore_degenerate=True)
    regridded = regridder(in_grid)
    return regridded

PE_mapunits_regridded = regrid_dataarray\
(PE_mapunits,grid_metrics_grid)

# then spatially integrate over 90S-50S to get Southern Ocean P-E time series. We need to redefine 
# the latitude subsetting function for the new grid 

def subset_bylatitude(data, south_bound, north_bound):
    data_SO = data.sel(y=slice(south_bound, north_bound))
    return data_SO

sftlf = grid_metrics_dict[grid_metrics_keys[0]].sftlf.squeeze().load()
areacella = grid_metrics_dict[grid_metrics_keys[1]].areacella.squeeze().load()

areacella_landmasked = areacella.where(sftlf==0,np.nan)

PE_mapunits_regridded_areaintegral = \
(subset_bylatitude(PE_mapunits_regridded, -90, -50)\
 * subset_bylatitude(areacella_landmasked, -90, -50))\
.sum(dim=('x', 'y')) 



In [24]:
# Finally, convert to units of Gt / yr, adjust the time dimension
# and save postprocessed data as a monthly time series 

os.chdir('/oak/stanford/groups/earlew/zkaufman/Archive_KaufmanGRL2025/postprocessed_analysis_notebooks')

PE_timeseriesunits_regridded_areaintegral = \
PE_mapunits_regridded_areaintegral * (1e-12) * (np.pi * 1e7)  # kg/s to Gt/yr

ERA5_PE_timeseries_postprocessed = PE_timeseriesunits_regridded_areaintegral.assign_coords(
    time=PE_timeseriesunits_regridded_areaintegral.indexes['time'].to_datetimeindex()
)


output_filename = 'ERA5_PE_timeseries_monthly.nc'
ERA5_PE_timeseries_postprocessed.to_netcdf(output_filename)
